In [2]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from datetime import datetime
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [40.79736, 41.78701, 30.1444, 25.7738],
	"longitude": [-73.97785, -87.77166, -97.66876, -80.1936],
	"start_date": "2016-01-01",
	"end_date": "2024-03-11",
	"daily": ["temperature_2m_max", "temperature_2m_min", "sunshine_duration", "precipitation_hours", "wind_speed_10m_max"],
}
responses = openmeteo.weather_api(url, params=params)

# TODO:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(3).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(4).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max

daily_dataframe = pd.DataFrame(data = daily_data)
# print(daily_dataframe)


Coordinates 40.808433532714844°N -74.0198974609375°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [9]:
df = daily_dataframe
df

,temperature_2m_max,temperature_2m_min,sunshine_duration,precipitation_hours,wind_speed_10m_max
date,,,,,
2016-01-01 00:00:00+00:00,6.730,1.880,27001.312500,0.0,21.077686
2016-01-02 00:00:00+00:00,4.380,-1.420,29190.664062,0.0,15.856356
2016-01-03 00:00:00+00:00,6.580,-0.970,29205.187500,0.0,19.083395
2016-01-04 00:00:00+00:00,2.080,-5.470,29141.408203,0.0,24.280659
2016-01-05 00:00:00+00:00,-1.220,-10.020,29472.777344,0.0,26.282465
...,...,...,...,...,...
2024-03-07 00:00:00+00:00,12.643,7.493,10800.000000,13.0,27.438046
2024-03-08 00:00:00+00:00,12.643,0.893,38151.398438,0.0,23.411074
2024-03-09 00:00:00+00:00,6.343,3.743,0.000000,6.0,25.952478


In [16]:
# Set the date as the index
# df['date'] = pd.to_datetime(df['date'])
# df.set_index('date', inplace=True)

# Assuming daily data, if not, resample as needed
df_daily = df.resample('D').mean()  # Resample to daily if not already

# Check for missing values and fill or interpolate
df_daily = df.interpolate(method='time')

# Decompose to observe seasonality, trend, and residuals
decomposition = seasonal_decompose(df_daily['temperature_2m_max'], model='additive')
decomposition.plot()
plt.show()

ValueError: This function does not handle missing values

In [6]:
# Example parameters: These should be tailored based on your dataset and analyses like ACF and PACF plots
p, d, q = 1, 1, 1
P, D, Q, m = 1, 1, 1, 365  # Assuming daily data with yearly seasonality

model = SARIMAX(df_daily['temperature_2m_max'],
                order=(p, d, q),
                seasonal_order=(P, D, Q, m))
                # enforce_stationarity=False,
                # enforce_invertibility=False)

results = model.fit()

# Summarize model
print(results.summary())

/opt/miniconda3/envs/cs542/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/miniconda3/envs/cs542/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.51260D+00    |proj g|=  6.52803D-01
